# How to match? 

## 0. Import libraries 

In [1]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.hypex import Matcher

warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/Users/20224761/PycharmProjects/matcher/lightautoml/ml_algo/dl_model.py:41: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/Users/20224761/PycharmProjects/matcher/lightautoml/text/nn_model.py:22: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/Users/20224761/PycharmProjects/matcher/lightautoml/text/dl_transformers.py:25: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,0,1,496,False
1,0,0,2,488,False
2,0,0,3,487,False
3,0,0,4,457,False
4,0,0,5,431,False


In [3]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,0,0,False,492.0,416.444444
1,2,0,False,500.5,412.111111
2,4,0,False,480.0,419.888889
3,6,0,False,488.5,433.111111
4,7,0,False,467.0,425.777778
...,...,...,...,...,...
5409,9995,0,False,457.0,407.222222
5410,9996,0,False,477.5,443.333333
5411,9997,0,False,483.5,416.000000
5412,9998,0,False,505.5,424.000000


In [4]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,0,0,0,492.0,416.444444,21,1,2
1,2,0,0,500.5,412.111111,65,0,1
2,4,0,0,480.0,419.888889,63,1,1
3,6,0,0,488.5,433.111111,51,1,2
4,7,0,0,467.0,425.777778,64,1,2


In [5]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [6]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [7]:
# Standard model with base parameters
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results, df_matched = model.estimate()

  0%|          | 0/5414 [00:00<?, ?it/s]

### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [8]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [9]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)
results, quality_results, df_matched = model.estimate()

  0%|          | 0/4 [00:00<?, ?it/s]

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [10]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)

In [11]:
features_importance = model.lama_feature_select()
features_importance

,Feature,Importance
0,pre_spends,38600.570068
3,industry,38600.570068
1,age,29652.719604
2,is_male,2970.189941


In [12]:
features = features_importance['Feature'].to_list()

In [13]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results, df_matched = model.estimate(features=features[:3])

  0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
df_matched

,index,pre_spends,age,is_male,industry,pre_spends_matched,age_matched,is_male_matched,index_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,4985,472.0,48,0,1,471.5,48.0,0.0,[1864],1,522.444444,421.888889,100.549753,1,0
1,4986,494.0,38,0,1,493.5,38.0,0.0,[1784],1,515.444444,423.222222,92.216419,1,0
2,4987,492.5,41,0,1,493.0,41.0,0.0,[927],1,529.666667,424.111111,105.561359,1,0
3,4988,479.0,59,1,1,478.0,59.0,0.0,[1128],1,513.333333,417.777778,95.543950,1,0
4,4989,490.5,52,0,1,490.0,52.0,0.0,[2196],1,515.555556,413.333333,102.216419,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4980,4980,502.5,45,0,2,501.5,42.0,1.0,[310],2,420.333333,523.666667,103.206430,0,1
4981,4981,477.5,49,1,2,478.0,48.0,1.0,[244],2,421.555556,521.222222,99.642777,0,1
4982,4982,488.0,69,1,2,488.5,69.0,0.0,[233],2,403.222222,519.555556,116.344380,0,1
4983,4983,505.5,40,1,2,503.0,39.0,1.0,[376],2,424.000000,533.222222,109.132051,0,1


## 3. Results  
### 3.1 ATE, ATT, ATC

In [15]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.972968,0.580032,0.0,99.836106,102.109830
ATC,100.963662,0.589903,0.0,99.807452,102.119872
ATT,101.081102,0.693178,0.0,99.722472,102.439732


### 3.2 SMD, PSI, KS-test, repeats

In [16]:
# matching quality result - SMD
model.quality_result

{'psi':         column_treated  anomaly_score_treated check_result_treated  \
 0          age_treated                   0.00                   OK   
 1     industry_treated                   0.00                   OK   
 2  post_spends_treated                  16.11                  NOK   
 3   pre_spends_treated                   0.01                   OK   
 
         column_untreated  anomaly_score_untreated check_result_untreated  
 0          age_untreated                     0.01                     OK  
 1     industry_untreated                     0.00                     OK  
 2  post_spends_untreated                     8.27                    NOK  
 3   pre_spends_untreated                     0.01                     OK  ,
 'ks_test':             match_control_to_treat  match_treat_to_control
 age                       1.000000                0.419063
 is_male                   0.320575                0.070995
 pre_spends                1.000000                0.510057,
 's

In [17]:
# matching quality result - PSI
model.quality_result['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,age_treated,0.00,OK,age_untreated,0.01,OK
1,industry_treated,0.00,OK,industry_untreated,0.00,OK
2,post_spends_treated,16.11,NOK,post_spends_untreated,8.27,NOK
3,pre_spends_treated,0.01,OK,pre_spends_untreated,0.01,OK


In [18]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
age,1.000000,0.419063
is_male,0.320575,0.070995
pre_spends,1.000000,0.510057


In [19]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.93, 'match_treat_to_control': 0.08}

### 3.3 Validation
validate result with placebo treatment or random feature or random subset
our new effect size is close to zero (it should be)

In [21]:
model.validate_result()

  0%|          | 0/10 [00:00<?, ?it/s]

{'post_spends': [100.61184307260706, 0.005630629641012241]}

In [22]:
model.save("test_model.pickle")

In [23]:
model2 = Matcher.load("test_model.pickle")

In [ ]:
model2.validate_result()